In [1]:
import tools
import cartesian
import racks
import calibration
import samples
import json
import importlib

# Creating dictionary with properties

## For 50 mL tubes

In [42]:
vol_vs_z = {
    0: 113.0, 
    5000: 95.0, 
    10000: 85, 
    15000: 76, 
    20000: 67, 
    25000: 58, 
    30000: 48, 
    35000: 40, 
    40000: 31, 
    45000: 23, 
    50000: 14    
}

In [43]:
sample_top_dz = {
    "50mL": 21.2
}

In [44]:
param = {
    'volume_vs_z': vol_vs_z,
    'sample_top_dz': sample_top_dz,
        }

In [45]:
with open('configs/50mL_tube_params.json', 'w') as fp:
    json.dump(param, fp)

## Eppendorf tube properties

In [121]:
vol_vs_z = {
    0: 38.45, 
    100: 32, 
    200: 28, 
    300: 26, 
    400: 24, 
    500: 22, 
    700: 18, 
    1000: 13, 
    1500: 5.8, 
    1700: 3
    }

In [122]:
sample_top_dz = {
    "eppendorf": 14.2,
    "eppendorf_magnetics": 10.5,
}

In [123]:
param = {
    'volume_vs_z': vol_vs_z,
    'sample_top_dz': sample_top_dz,
        }

In [124]:
with open('configs/eppendorf_tube_params.json', 'w') as fp:
    json.dump(param, fp)

# Working with sample class

In [105]:
importlib.reload(samples)

<module 'samples' from 'c:\\Users\\sergi\\Dropbox\\workspace\\arnielib\\samples.py'>

In [106]:
t50 = samples.sample(sample_name='t50_1', sample_type='50mL_tube')

In [107]:
t50.sample_data

{'sample_name': 't50_1', 'sample_type': '50mL_tube'}

In [108]:
t50.params

{'volume_vs_z': {'0': 113.0,
  '5000': 95.0,
  '10000': 85,
  '15000': 76,
  '20000': 67,
  '25000': 58,
  '30000': 48,
  '35000': 40,
  '40000': 31,
  '45000': 23,
  '50000': 14},
 'sample_top_dz': {'50mL': 21.2}}

In [46]:
importlib.reload(racks)

<module 'racks' from 'c:\\Users\\sergi\\Dropbox\\workspace\\arnielib\\racks.py'>

In [5]:
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [6]:
r50.rack_data

{'name': '50mL_rack_1',
 'n_x': 3,
 'n_y': 2,
 'type': '50mL',
 'position': [538.75, 286.7, 508.95],
 'pos_stalagmyte': [92.8, 66.2, 490.55]}

In [7]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [110]:
ar.home()

In [9]:
stp = tools.stationary_touch_probe(ar)

In [10]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [140]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [12]:
ar.move(z=400)

In [13]:
p1000_tip_rack = racks.rack(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [112]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [113]:
ar.move(z=300)

## Emulating pipetting procedure

In [109]:
t50.place(r50, 2, 0)

In [17]:
sx, sy = r50.calcWellXY(2, 0)

In [18]:
ar.move(x=sx, y=sy)

In [19]:
z_above_calibr = t50.params['sample_top_dz']['50mL_rack']

In [20]:
xr, yr, zr = r50.calcWorkingPosition(2, 0, p1000)

In [21]:
xr, yr, zr

(588.45, 261.9, 369.75)

In [22]:
ar.move(z=zr)

In [28]:
t50.getSampleCenterXY()

(588.75, 261.7)

In [35]:
t50.getSampleCenterXY(p1000)

(588.45, 261.9)

In [56]:
t50.sampleDepthToZ(0, p1000)

348.55

In [57]:
ar.move(z=348.55)

In [38]:
t50.sample_data

{'sample_name': 't50_1',
 'sample_type': '50mL_tube',
 'rack': <racks.rack at 0x1caef27fb08>,
 'x_well': 2,
 'y_well': 0}

In [40]:
t50.params['sample_top_dz'][]

{'50mL_rack': 21.2}

In [41]:
r50.rack_data

{'name': '50mL_rack_1',
 'n_x': 3,
 'n_y': 2,
 'type': '50mL',
 'position': [538.75, 286.7, 508.95],
 'pos_stalagmyte': [92.8, 66.2, 490.55]}

In [51]:
v_vs_z = t50.params['volume_vs_z']

In [19]:
v_vs_z.keys()

dict_keys(['0', '5000', '10000', '15000', '20000', '25000', '30000', '35000', '40000', '45000', '50000'])

In [22]:
requested_vol = 38000

In [25]:
diff_dict = {}
for key in v_vs_z.keys():
    standard_vol = float(key)
    diff = abs(standard_vol - requested_vol)
    diff_dict[key] = diff

In [29]:
diff_dict

{'0': 38000.0,
 '5000': 33000.0,
 '10000': 28000.0,
 '15000': 23000.0,
 '20000': 18000.0,
 '25000': 13000.0,
 '30000': 8000.0,
 '35000': 3000.0,
 '40000': 2000.0,
 '45000': 7000.0,
 '50000': 12000.0}

In [43]:
diff_val_list = list(diff_dict.values())

In [44]:
temp = min(diff_val_list)

In [45]:
diff_val_list.remove(temp)

In [46]:
temp2 = min(diff_val_list)

In [47]:
temp, temp2

(2000.0, 3000.0)

In [52]:
res1 = [key for key in diff_dict if diff_dict[key] == temp][0]
res2 = [key for key in diff_dict if diff_dict[key] == temp2][0]

In [53]:
res1, res2

('40000', '35000')

In [58]:
p = [float(res1), v_vs_z[res1]]
q = [float(res2), v_vs_z[res2]]

In [59]:
p, q

([40000.0, 31], [35000.0, 40])

In [60]:
a = q[1] - p[1]
b = p[0] - q[0]
c = a * (p[0]) + b * (p[1])

In [61]:
a, b, c

(9, 5000.0, 515000.0)

In [66]:
(c - a * 39000) / b

32.8

In [76]:
def calcZFromVolume(volume, vol_vs_z_dict):
    diff_dict = {}
    for key in vol_vs_z_dict.keys():
        standard_vol = float(key)
        diff = abs(standard_vol - volume)
        diff_dict[key] = diff
    diff_val_list = list(diff_dict.values())
    temp = min(diff_val_list)
    diff_val_list.remove(temp)
    temp2 = min(diff_val_list)
    standard_vol_1 = [key for key in diff_dict if diff_dict[key] == temp][0]
    standard_vol_2 = [key for key in diff_dict if diff_dict[key] == temp2][0]
    standard_z_1 = vol_vs_z_dict[standard_vol_1]
    standard_z_2 = vol_vs_z_dict[standard_vol_2]

    p = [float(standard_vol_1), standard_z_1]
    q = [float(standard_vol_2), standard_z_2]
    
    a = q[1] - p[1]
    b = p[0] - q[0]
    c = a * (p[0]) + b * (p[1])
    
    desired_z = (c - a * volume) / b
    return desired_z

In [82]:
calcZFromVolume(7000, v_vs_z)

91.0

In [81]:
v_vs_z

{'0': 113.0,
 '5000': 95.0,
 '10000': 85,
 '15000': 76,
 '20000': 67,
 '25000': 58,
 '30000': 48,
 '35000': 40,
 '40000': 31,
 '45000': 23,
 '50000': 14}

In [96]:
t50.sampleVolToZ(50000, p1000)

362.45

In [114]:
x, y = t50.getSampleCenterXY(p1000)

In [115]:
ar.move(x=x, y=y)

In [97]:
ar.move(z=362.45)

In [117]:
z = t50.samplePercentToZ(0.5, p1000)

In [118]:
ar.move(z=z)

In [119]:
ar.move(z=300)

In [120]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-20)

In [65]:
p1000.returnTool()

In [103]:
ar.home()

# Eppendorf tube

In [132]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')

In [133]:
ep1.rack_data

{'name': 'eppendorf_1',
 'n_x': 0,
 'n_y': 3,
 'type': 'eppendorf',
 'position': [91.35, 395.29999999999995, 508.9],
 'pos_stalagmyte': [92.85, 66.15, 490.7]}

In [134]:
t_ep = samples.sample(sample_name='t_ep_1', sample_type='eppendorf_tube')

In [135]:
t_ep.place(ep1, 0, 0)

In [136]:
t_ep.sample_data

{'sample_name': 't_ep_1',
 'sample_type': 'eppendorf_tube',
 'rack': <racks.rack at 0x1d8fd5976c8>,
 'x_well': 0,
 'y_well': 0}

In [141]:
p1000.pickUpTip(p1000_tip_rack, 0, 0, raise_z=400)

In [142]:
ar.move(z=300)

In [143]:
x, y = t_ep.getSampleCenterXY(p1000)

In [144]:
ar.move(x=x, y=y)

In [145]:
z = t_ep.sampleVolToZ(200, p1000)

In [146]:
ar.move(z=z)

In [147]:
p1000.dropOffTipToPosition(p1000_tip_rack, 0, 0, raise_z=300, dropoff_dz=-30)

In [148]:
p1000.returnTool()

In [149]:
ar.home()